# Lab assignment: the hunger games

<table><tr>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/breakfast.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/hamburger.jpg" style="width:300px;height:300px;"></td>
    <td><img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/fruits.jpg" style="width:300px;height:300px;"></td>
</tr></table>

In this assignment we will face a challenging image classification problem, building a deep learning model that is able to classify different kinds of foods. Let the hunger games begin!

## Guidelines

Throughout this notebook you will find empty cells that you will need to fill with your own code. Follow the instructions in the notebook and pay special attention to the following symbols.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
You will need to solve a question by writing your own code or answer in the cell immediately below or in a different file, as instructed.</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
This is a hint or useful observation that can help you solve this assignment. You should pay attention to these hints to better understand the assignment.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
This is an advanced exercise that can help you gain a deeper knowledge into the topic. Good luck!</font>

***

To avoid missing packages and compatibility issues you should run this notebook under one of the [recommended Deep Learning environment files](https://github.com/albarji/teaching-environments/tree/master/deeplearning), or make use of [Google Colaboratory](https://colab.research.google.com/). If you use Colaboratory make sure to [activate GPU support](https://colab.research.google.com/notebooks/gpu.ipynb).

The following code will embed any plots into the notebook instead of generating a new window:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Lastly, if you need any help on the usage of a Python function you can place the writing cursor over its name and press Caps+Shift to produce a pop-out with related documentation. This will only work inside code cells. 

Let's go!

## Data acquisition

We will use a food images dataset available at [Kaggle](https://www.kaggle.com/trolukovich/food11-image-dataset). To download it you will need to create a user account in Kaggle, and obtain your API credential by following the instructions on [this section](https://www.kaggle.com/trolukovich/food11-image-dataset). Once you have your credentials JSON file, you can inform this notebook of them by setting the appropriate enviroment variables, as follows

    import os

    os.environ["KAGGLE_USERNAME"] = "YOUR KAGGLE USERNAME HERE"
    os.environ["KAGGLE_KEY"] = "YOUR KAGGLE KEY HERE"
    
Once this is done, you will be able to download the dataset to this computer using the command

    !kaggle datasets download trolukovich/food11-image-dataset --unzip -p YOUR_LOCAL_FOLDER
    
where you should write a valid directory in your computer in "YOUR_LOCAL_FOLDER". If you are fine with downloading the data in the same folder as this notebook, just skip the `-p YOUR_LOCAL_FOLDER` part of the command.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Create a Kaggle account, obtain your credentials, and use the cell below to declare your Kaggle username and key variables, and to download the dataset to a local folder.
    
These credentials should remain secret to you. Remember to delete them before submitting this notebook!
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Take now a look into the folder where you downloaded the data. You will find it is made up of three subfolders:

* **training**, containing the images to use to train the model.
* **validation**, containing additional images you could use as more training data, or for some kind of validation strategy such as Early Stopping.
* **evaluation**, containing the images you must use to test your model. Images in this folder can **only** be used to measure the model performance after the training procedure is completed.

Furthermore, within each one of these folders you will find one folder for each one of the 11 classes in this problem:

* Bread
* Dairy product
* Dessert
* Egg
* Fried food
* Meat
* Noodles-Pasta
* Rice
* Seafood
* Soup
* Vegetable-Fruit

This is a standard structure for organizing image datasets: one folder per class. To easen the following data processing steps, let us define some variables telling us where the data is located.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create variables <b>TRAINDIR</b>, <b>VALDIR</b> and <b>TESTDIR</b> with the paths to the folders with the training, validation and evaluation data, respectively.
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Let's plot a random sample of training images from each class, using the ipyplot package. If you are running this notebook in Google Colab, you will need to install this package first with

    !pip install ipyplot

You can inspect each class by clicking the different tabs in the interface that will appear when running the following cell.

In [ ]:
from glob import glob
import ipyplot
import numpy as np

all_images = glob(f"{TRAINDIR}/*/*.jpg")  # Get all image paths
np.random.shuffle(all_images)  # Randomize to show different images each run
all_labels = [f.split("/")[-2] for f in all_images]  # Extract class names from path

ipyplot.plot_class_tabs(all_images, all_labels, max_imgs_per_tab=6, img_width=300, force_b64=True)

### Class reduction

To make the problem more approachable for this exercise, we will focus on just six classes: `Bread`, `Dairy product`, `Dessert`, `Egg`, `Fried food` and `Meat`. To do so, we will delete from the downloaded data the folders from other classes.

In [ ]:
from glob import glob
import os

valid_classes = {"Bread", "Dairy product", "Dessert", "Egg", "Fried food", "Meat"}
datasets = {TRAINDIR, VALDIR, TESTDIR}

for dataset in datasets:
    for classdir in glob(f"{dataset}/*"):  # Find subfolders with classes
        if classdir.split("/")[-1] not in valid_classes:  # Ignore those in valid_classes
            print(f"Deleting {classdir}...")
            for fname in glob(f"{classdir}/*.jpg"):  # Remove each image file
                os.remove(fname)
            os.rmdir(classdir)  # Remove folder

## Image processing from files

This dataset of images is large, with images of larger resolution than in the tutorial MNIST problem, each one having different sizes and image ratios. Also, while for MNIST we had a keras function that prepared the dataset for us, this time we will need to do some loading and image processing work.

A convenient way to do this work is through the use of Keras `image_dataset_from_directory` function. This function creates a TensorFlow `Dataset` with the images in the directory, loading images dynamically only when the neural network needs to use them, and also allowing us to specify some useful preprocessing options.

For example, we can create such a `Dataset` with the data in the training folder:

In [ ]:
from keras.preprocessing import image_dataset_from_directory

image_size = 32
batch_size = 64

train_dataset = image_dataset_from_directory(
    TRAINDIR, 
    image_size = (image_size, image_size),
    batch_size = batch_size, 
    label_mode = 'categorical'
)

Note the parameters used to configure the dataset:

* The **directory** from which to load the images.
* An **image size** that will be used to resize all the images to a common resolution, here 32x32.
* The **size of the batches** of images to be generated. Note we define this parameter here instead in the network fit step, as the `Dataset` will make use of this information to keep in memory only a few batches of images at the same time in order to save memory.
* The **label mode**, that is, the encoding used for the labels. `categorical` means we will use one-hot encoding.

A `Dataset` object works like a Python generator, which means we can iterate over it to obtain batches of processed images. For instance, to get the first batch

In [ ]:
for X_batch, y_batch in train_dataset:
    print(f"Shape of input batch: {X_batch.shape}")
    print(f"Shape of output batch: {y_batch.shape}")
    print(f"Input batch:\n{X_batch}")
    print(f"Output batch:\n{y_batch}")
    break

We can see that indeed the generator produces a tensor of the appropriate dimensions with the inputs for the neural network, and that the outputs have also been properly codified in one-hot form. However, there is still an issue with the data: the pixel values are in the range [0, 255], which might produce training problems. We will solve this later in the network definition. For now, let's move on and prepare a funcion that builds the training, validation and test datasets.

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Create a function <b>create_datasets</b> that receives the following parameters:
    <ul>
      <li><b>traindir</b>: the directory where training images are located.</li>
      <li><b>valdir</b>: the directory where validation images are located.</li>
      <li><b>testdir</b>: the directory where test images are located.</li>
      <li><b>image_size</b>: the size that will be used to resize all the images to a common resolution.</li>
      <li><b>batch_size</b>: the size of the batches of images to be generated.</li>
    </ul>
    The function must create datasets for the training, validation and test directories, and return the three datasets created.

    return train_dataset, val_dataset, test_dataset
</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

Let's test if the function you just implemented works correctly

In [ ]:
import tensorflow as tf

train_dataset, val_dataset, test_dataset = create_datasets(TRAINDIR, VALDIR, TESTDIR, image_size=32, batch_size=64)

# Test whether all returned objects are valid Tensorflow datasets
assert isinstance(train_dataset, tf.data.Dataset)
assert isinstance(val_dataset, tf.data.Dataset)
assert isinstance(test_dataset, tf.data.Dataset)

Now that we have our datasets we can train a deep learning model using them! For illustration purposes, let's build an extremely simple convolutional network. Note how we have added a special pre-processing layer `Rescaling` that takes care of normalizing the data to the range [0, 1].

Be careful! This network will work, but has some flaws in its design you might want to fix in the network you will desing later in this notebook.

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.experimental.preprocessing import Rescaling

model = Sequential()
model.add(Rescaling(scale=1./255, input_shape=(image_size, image_size, 3)))
model.add(Convolution2D(4, 3, activation='linear'))
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])

The `fit` method of a Keras model can receive a `Dataset` with training data, instead of a pair of tensors with (inputs, outputs). Since when building the `Dataset` we already specified the batch size, we don't need to do it now.

In [ ]:
model.fit(train_dataset, epochs=1)

Similarly, we can evaluate the performance of our model over our test `Dataset` as follows

In [ ]:
loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

The accuracy might seem poor, but take into account we have used a very simple model and this problem has 6 classes. Will you be able to do better?

## Building your network

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Design a neural network that maximizes the accuracy over the test data. You can use the training and validation datasets for anything you like, but you can <b>only</b> use the test data to evaluate the model performance. You should obtain a network able to attain at least 40% accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Make use of all the tricks you learned from previous notebooks: convolutional + pooling layers, ReLU activations, dropout... also make sure to use a good optimizer with an adequate loss function, as well as the correct activation for the output layer.
- Configuring the datasets to load the images with a larger size can significantly improve your performance. But be careful, you can also run out of memory (CUDA memory error) if they become too large! Note that for this problem a size larger than 256 might be too large.
- Start with networks with a small number of parameters, so you are able to check fast how well they work. Then you can make your network larger in three directions: larger input images, more layers and more kernels per convolutional layer or units per dense layer. If you use larger images make sure to add more Convolution+Pooling layers, so that only very small images (less than 10x10 pixels) arrive at the Flatten layer.
- If you see large differences in loss between your training data and your validation or test data, try increasing the Dropout probabilities, especially for the Dense layers.
- Make use of that validation data! For instance, use an <a href="https://keras.io/api/callbacks/early_stopping/">**EarlyStopping strategy**</a> to monitor the loss of these validation data, and stop when training when after a number of iterations such loss has not decreased. Configuring the EarlyStopping to restore the best weights found in the optimization is also useful.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
Other advanced strategies you can try are:

- Use **image augmentation techniques** to artifically create new training images. To do so, explore the rest of layers available in the <a href="https://keras.io/api/layers/preprocessing_layers/image_preprocessing/">Keras Image Preprocessing module</a>.
- Use <a href="https://keras.io/api/layers/normalization_layers/batch_normalization/">BatchNormalization</a> layers to improve the optimization procedure.
    
It you use all the tricks, it is possible to obtain more than 60% accuracy in the test set.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Transfer learning

While designing your own network might produce some nice results, it is generally better to transfer the knowledge available in a pre-trained network. This not only can produce better results, but also saves a lot of design time. The [Keras Applications](https://keras.io/api/applications/) module contains several network designs ready to use. For instance, to exploit the famous VGG16 network we do

In [ ]:
from keras.applications import VGG16

vgg16_model = VGG16(include_top=False, input_shape=(image_size, image_size, 3))

By default all Keras Applications networks are preloaded with the weights that were obtained from training the network over the [ImageNet dataset](http://www.image-net.org/). To adapt the network to our problem we need to specify the shape of our input images, and also remove the output layers (top) of the original network, since we have a different number of classes.

Now, how do we do transfer learning over this network? We will show here how to implement the bottleneck features strategy. First, we will mark the VGG16 model as **non-trainable**, so that the weights remain frozen

In [ ]:
vgg16_model.trainable = False

Now we will build a neural network that includes the VGG16 model as one of its "layers". Since the VGG16 was trained with an specific way of normalizing the images, we will need to normalize our images in the same way. Conveniently, Keras also provides a function for doing VGG16-style normalization.

In [ ]:
from keras.applications.vgg16 import preprocess_input

We can try this with some image ir our dataset

In [ ]:
for X_batch, _ in train_dataset:
    break
    
print(f"Before normalizing: {X_batch[0, :3, :3, :]}")
print(f"After normalizing: {preprocess_input(X_batch)[0, :3, :3, :]}")

The normalization required by VGG16 involves swapping the order of color channels (RGB -> BGR) and substracting the mean values over the ImageNet dataset for each color channel separately. Fortunately the `preprocess_input` function does all the work for us. Furthermore, we can plug this function as the first layer of our network, similarly to the `Rescaling` we used before. We can do this with a `Lambda` layer, which allows building a layer out of any (differentiable!) function. Let's start our model with this layer.

In [ ]:
from keras.layers import Lambda

model = Sequential()
model.add(Lambda(preprocess_input, input_shape=(image_size, image_size, 3)))

After this, we can add the whole VGG16 network as a layer, and our custom trainable layers after it. Note this is an overly simple model with some mistakes introduced; a real transfer learning network would have a better design.

In [ ]:
model.add(vgg16_model)
model.add(Flatten())
model.add(Dense(6, activation='sigmoid'))
model.summary()

Notice how in the model summary we can see the whole network has millions of parameters, but since we have frozen the VGG16 part, only a few thousand parameters will be trained: those in the Dense layer.

We can now compile and train this model in the usual way

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=["accuracy"])
model.fit(train_dataset, epochs=1)

loss, acc = model.evaluate(test_dataset)
print(f"Loss {loss:.3}, accuracy {acc:.1%}")

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
    Using the bottleneck strategy, implement a network doing transfer learning from VGG16. If you do it properly, you should be able to obtain better results than with your previously designed network, at least a 80% of accuracy over the test set.
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/exclamation.png" height="80" width="80" style="float: right;"/>

***
<font color=#2655ad>
    
Some tips and strategies that can help you optimize your network design:

    
- Include one or more Dense layers with the appropriate activation functions before the output layer.
- Try using a [GlobalAveragePooling layer](https://keras.io/api/layers/pooling_layers/global_average_pooling2d/) instead of a Flatten layer. This layer computes an average of all pixel values for each channel, and sometimes performs better than a regular Flatten.
- And remember all the tricks from the previous exercise!
</font>

***

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/pro.png" height="80" width="80" style="float: right;"/>

***
<font color=#259b4c>
    
You can further improve the network accuracy with the following ideas

- Use the PRO strategies from the previous exercise.
- Try other pre-trained networks from <a href="https://keras.io/api/applications/">Keras Applications</a>, such as ResNet, Xception or EfficientNet.
- Use more advanced transfer learning strategies, like fine-tuning or a combination of bottleneck features and fine-tuning.
   
If you use all the tricks, it is possible to obtain more than 90% accuracy in the test set.

</font>

***

In [ ]:
####### INSERT YOUR CODE HERE

## Summary of results

<img src="https://albarji-labs-materials.s3-eu-west-1.amazonaws.com/question.png" height="80" width="80" style="float: right;"/>

***

<font color=#ad3e26>
Write in the following cell a short report explaining what network designs you tried, and what test accuracies you obtained. What worked and what didn't? What have you learned from this exercise?
</font>

***

Example results table

|Image processing|Neural network model|Training strategy|Test accuracy|
|----------------|--------------------|-----------------|-------------|
|Size 32x32, batch size 16|Convolutional(32) + Flatten + Dense(64)|Train from scratch|xx%|
|Size 64x64, batch size 32|VGG16 + Flatten + Dense(32)|Bottleneck features|yy%|
|...|...|...|...|